In [21]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('../data/checking-logs.sqlite')


In [22]:
ab_query = """
WITH base AS (
    SELECT t.uid,
           CASE WHEN t.first_commit_ts < t.first_view_ts THEN 'before' ELSE 'after' END AS time,
           (julianday(t.first_commit_ts) - julianday(datetime(d.deadlines,'unixepoch'))) * 24.0 AS diff_hours
    FROM {table} t
    JOIN deadlines d ON t.labname = d.labs
    WHERE t.labname <> 'project1'
),
filtered AS (
    SELECT * 
    FROM base
    WHERE uid IN (
        SELECT uid 
        FROM base
        GROUP BY uid
        HAVING SUM(time='before') > 0 AND SUM(time='after') > 0
    )
),
uid_avg AS (
    SELECT uid, time, AVG(diff_hours) AS uid_avg_diff
    FROM filtered
    GROUP BY uid, time
)
SELECT time, AVG(uid_avg_diff) AS avg_diff
FROM uid_avg
GROUP BY time;
"""
test_results    = pd.read_sql(ab_query.format(table='test'),    conn)
control_results = pd.read_sql(ab_query.format(table='control'), conn)

test_results, control_results


(     time    avg_diff
 0   after -100.178032
 1  before  -66.679398,
      time   avg_diff
 0   after -99.803422
 1  before -98.467698)

In [23]:
conn.close()
